#Global Climate Trends
##Analysis

In [18]:
import pandas as pd
import os
import csv

In [31]:
# Path to collect data from the Resources folder
folder_path = os.path.join(".", "Sorted")

# List all files in the specified directory
files = os.listdir(folder_path)

# Filter only CSV files
csv_files = [file for file in files if file.endswith('.csv')]
# Verifying is files are being read
    # print(csv_files)
